In [2]:
import pandas as pd

In [3]:
""" Get stuff out of Netfile v2 API
"""
from pprint import PrettyPrinter
from pathlib import Path
import os
import requests

BASE_URL = 'https://netfile.com/api/campaign'
CONTRIBUTION_FORM = 'F460A'
EXPENDITURE_FORM = 'F460E'

PARAMS = { 'aid': 'COAK' }

def get_auth_from_env_file(filename: str='.env'):
    """ Split .env file on newline and look for API_KEY and API_SECRET
        Return their values as a tuple
    """
    env_file=Path(filename)
    auth_keys = [ 'API_KEY', 'API_SECRET' ]
    if env_file.exists():
        auth = tuple( v for _, v in sorted([
            ln.split('=') for ln in
            env_file.read_text(encoding='utf8').strip().split('\n')
            if ln.startswith(auth_keys[0]) or ln.startswith(auth_keys[1])
        ], key=lambda ln: auth_keys.index(ln[0])))
    else:
        auth=tuple(os.environ[key] for key in auth_keys)
            
    return auth

AUTH=get_auth_from_env_file()

pp = PrettyPrinter()

def get_filing(offset=0):
    """ Get a filing
    """
    url = f'{BASE_URL}/filing/v101/filings'

    params = { **PARAMS }
    if offset > 0:
        params['offset'] = offset

    res = requests.get(url, params=params, auth=AUTH)
    body = res.json()
    results = body.pop('results')

    return results, body
def get_form(form,offset=0):
    """ Get a filing
    """
    url = f'{BASE_URL}/filing/v101/filings?Limit=100000&SpecificationForm={form}'

    params = { **PARAMS }
    if offset > 0:
        params['offset'] = offset

    res = requests.get(url, params=params, auth=AUTH)
    body = res.json()
    results = body.pop('results')

    return results, body
def get_filer(filer_nid):
    """ Get one filer
    """
    url = f'{BASE_URL}/filer/v101/filers?'

    res = requests.get(url, params={ **PARAMS, 'filerNid': filer_nid }, auth=AUTH)
    body = res.json()

    return body['results']
def list_filers():
    """ Get all the elections
    """
    url = f'{BASE_URL}/filer/v101/filers?Limit=100000'

    res = requests.get(url, params=PARAMS, auth=AUTH)
    body = res.json()

    return body['results']
def list_elections_influences(id):
    """ Get all the elections
    """
    url = f'{BASE_URL}/election/v101/election-influences?Limit=100000&ElectionNid={id}'

    res = requests.get(url, params=PARAMS, auth=AUTH)
    body = res.json()

    return body['results']
def list_elections():
    """ Get all the elections
    """
    url = f'{BASE_URL}/election/v101/elections?Limit=100000'

    res = requests.get(url, params=PARAMS, auth=AUTH)
    body = res.json()

    return body['results']

In [4]:
filing=get_filing()

In [5]:
filers_response=list_filers()

In [6]:
[item for item in filers_response if item['registrations']]

[{'filerNid': '202521854',
  'visibilityLevel': 'Staff',
  'filerName': 'Nick Resnick for Oakland School Board 2022 District 4',
  'candidateName': 'Resnick, Nick',
  'aid': 'COAK',
  'adsid': 163882,
  'isTerminated': True,
  'committeeTypes': ['Candidate or Officeholder'],
  'nameHistory': ['Resnick, Nick'],
  'addressList': [{'addressTypes': 'Disclosure, Mailing',
    'line1': '4146 Whittle Ave',
    'line2': '',
    'city': 'Oakland',
    'state': 'CA',
    'zip': '94602'}],
  'phoneList': [{'phoneType': 'Other', 'number': '5103066472'}],
  'emailList': [{'address': 'resnick4@gmail.com',
    'emailType': 'Notification'}],
  'statusItemList': [{'status': 'ACTIVE',
    'date': '2022-03-04T17:28:18.4317574+00:00'},
   {'status': 'TERMINATED', 'date': '2024-02-01T06:08:35.2880576+00:00'}],
  'registrations': {'CA SOS': '1446325'},
  'officers': [{'officerName': 'Guzman, Ben',
    'position': 'Treasurer',
    'email': {'address': 'jkdguz101@gmail.com', 'emailType': 'Notification'},
    

In [7]:
elections=list_elections()

In [8]:
elections

[{'electionNid': '200879019',
  'aid': 'COAK',
  'electionDate': '2024-11-05',
  'electionCaption': '11/5/2024 - General',
  'isPublic': True,
  'electionCodes': 'General',
  'measures': [],
  'seats': [{'seatNid': '207972556',
    'electionNid': '200879019',
    'seatOfficeNid': '121709381',
    'seatCaption': 'City Attorney - City of Oakland',
    'officeName': 'City Attorney - City of Oakland'},
   {'seatNid': '207972521',
    'electionNid': '200879019',
    'seatOfficeNid': '121709393',
    'seatCaption': 'City Council - City of Oakland - 1',
    'officeName': 'City Council - City of Oakland - 1'},
   {'seatNid': '207972552',
    'electionNid': '200879019',
    'seatOfficeNid': '121709414',
    'seatCaption': 'City Council - City of Oakland - 3',
    'officeName': 'City Council - City of Oakland - 3'},
   {'seatNid': '207972554',
    'electionNid': '200879019',
    'seatOfficeNid': '121709440',
    'seatCaption': 'City Council - City of Oakland - 5',
    'officeName': 'City Council

In [9]:
form410s=get_form('FPPC410')
form410s=form410s[0]
form410={'filerNid':[],'fppc_id':[]}
for form in form410s:
    form410['filerNid'].append(form['filerMeta']['filerId'])
    form410['fppc_id'].append(form.get('filerMeta',{}).get('strings',{}).get('Registration_CA SOS',None))
df_410=pd.DataFrame(form410)
df_410.drop_duplicates()

,filerNid,fppc_id
0,209987768,Pending
1,202521854,1446325
3,208595769,1462967
4,204523642,Pending
5,204218262,1449393
...,...,...
652,121706423,1304614
653,121706989,1304551
654,121707017,1304075
655,121706871,1303555


In [10]:
form501s=get_form('FPPC501')
form501s=form501s[0]
form501={'filerNid':[],'fppc_id':[]}
for form in form501s:
    form501['filerNid'].append(form['filerMeta']['filerId'])
    form501['fppc_id'].append(form.get('filerMeta',{}).get('strings',{}).get('Registration_CA SOS',None))
df_501=pd.DataFrame(form501)
df_501.drop_duplicates()
form501s

[{'apiVersion': 'V101',
  'filingNid': 'f822ccd8-f064-456c-8502-9ceed9b15ce7',
  'filingVersion': 1,
  'visibilityLevel': 'Staff',
  'filingActivityType': 'FiledOriginal',
  'calculatedDate': None,
  'filingAuthority': 'NetFileAgencyAdmin',
  'originalFilingId': '209634370',
  'specificationRef': {'org': 'CAL', 'name': 'FPPC501', 'version': '2.0'},
  'filingMeta': {'specificationKey': 'CAL:FPPC501:2.0',
   'issuedFilingId': '209634370',
   'mechanismType': 'Paper',
   'formId': 'c109498d-9246-4be5-9090-d744cc887300',
   'legalFilingDateTime': '2024-01-24T08:00:00+00:00',
   'legalFilingDate': '2024-01-24',
   'startDate': None,
   'endDate': None,
   'amendmentType': 'None',
   'amendmentSequence': None,
   'amendmentDescription': None,
   'reportNumber': '',
   'vendorName': 'NetFile',
   'vendorVersion': '1.0',
   'vendorTrackingId': None,
   'notes': {},
   'strings': {'FormGroupId': '840b5cf5-d3e3-45d2-81f6-f717da4142ec',
    'SAN': None},
   'numbers': {'FormToolHash': 0},
   'dat

In [11]:
forms_df=pd.concat([df_410,df_501]).drop_duplicates()
forms_df

,filerNid,fppc_id
0,209987768,Pending
1,202521854,1446325
3,208595769,1462967
4,204523642,Pending
5,204218262,1449393
...,...,...
280,130590233,1345618
283,121707107,1261569
286,150316345,None
292,132415352,1345683


In [12]:
fppc={'filerNid':[], 'fppc_id':[]}
for item in filers_response:
    fppc['filerNid'].append(item.get('filerNid',{}))
    fppc['fppc_id'].append(item.get('registrations',{}).get('CA SOS',None))
fppc_df=pd.DataFrame(fppc)
fppc_df.drop_duplicates()

,filerNid,fppc_id
0,210215097,None
1,210133373,None
2,210131642,None
3,210122806,None
4,210122167,None
...,...,...
1109,121706546,1304061
1110,121706516,1245211
1111,121706486,931297
1112,121706456,1303541


In [13]:
merge_df=forms_df.merge(fppc_df, on='filerNid')
merge_df=merge_df[['filerNid','fppc_id_y']]
merge_df=merge_df.rename(columns={'fppc_id_y':'fppc_id'})
merge_df

,filerNid,fppc_id
0,209987768,Pending
1,202521854,1446325
2,202521854,1446325
3,208595769,1462967
4,208595769,1462967
...,...,...
488,130590233,1345618
489,121707107,1261569
490,150316345,None
491,132415352,1345683


In [14]:
election_list=[]
previous_df=pd.DataFrame()
for election in elections:
    candidates=election['candidates']
    seats=election['seats']
    election_name=election['electionCaption']
    electionNid=election['electionNid']
    election_key={'election_name':election_name, 'electionNid':electionNid}
    election_list.append(election_key)
    if candidates and seats:
        seat_df=pd.DataFrame(seats)
        candidate_df=pd.DataFrame(candidates)
        merge_df=candidate_df.merge(seat_df, on='seatNid')
        current_df=merge_df[['candidateNid','candidateName','seatNid','officeName','electionNid']]
        previous_df=pd.concat([previous_df,current_df],ignore_index=True)
    #merge_df=merge_df.merge(election_key, on='electionNid')
election_df=pd.DataFrame(election_list)
final_df=previous_df.merge(election_df, on='electionNid')
final_df


,candidateNid,candidateName,seatNid,officeName,electionNid,election_name
0,180842909,"Davis, Benjamin ""Sam""",207972561,Director - Oakland Unified School District - 1,200879019,11/5/2024 - General
1,208539738,"Dunbar, Donnel C",207972552,City Council - City of Oakland - 3,200879019,11/5/2024 - General
2,161700164,"Fife, Carroll",207972552,City Council - City of Oakland - 3,200879019,11/5/2024 - General
3,208555055,"Hirsch, Shan M",207972552,City Council - City of Oakland - 3,200879019,11/5/2024 - General
4,208504223,"Logan, Warren",207972552,City Council - City of Oakland - 3,200879019,11/5/2024 - General
...,...,...,...,...,...,...
355,121707619,"Kernighan, Patricia",149465894,City Council - City of Oakland - 2,125693758,6/6/2006 - Special
356,133250767,"Drake, Pamela",133250723,City Council - City of Oakland - 2,133250667,5/17/2005 - Special
357,121710764,"Kakishiba, David",133250723,City Council - City of Oakland - 2,133250667,5/17/2005 - Special
358,121707619,"Kernighan, Patricia",133250723,City Council - City of Oakland - 2,133250667,5/17/2005 - Special


In [15]:
election_ids=list(set(final_df['electionNid'].to_list()))
previous_df=pd.DataFrame()
for id in election_ids:
    influences=list_elections_influences(id)
    influences_dic={'filerNid': [],'electionNid': [],'seatNid': [],'candidateNid': [],'committeeName':[],'election_name': []} # ,'isWinner':[],'isIncumbent':[]
    for candidate in influences:   
        influences_dic['filerNid'].append(candidate.get('filerNid', 'None'))
        influences_dic['election_name'].append(candidate.get('electionCaption', 'None'))
        influences_dic['committeeName'].append(candidate.get('committeeName', 'None'))
        influences_dic['electionNid'].append(candidate.get('electionNid', 'None'))
        influences_dic['seatNid'].append(candidate.get('seatNid', 'None'))
        influences_dic['candidateNid'].append(candidate.get('candidateNid', 'None'))
        # influences_dic['isWinner'].append(candidate.get('candidate',{}).get('isWinner', 'None'))
        # influences_dic['isIncumbent'].append(candidate.get('candidate',{}).get('isIncumbent', 'None'))
        current_df=pd.DataFrame(influences_dic)
        current_df=current_df
        previous_df=pd.concat([previous_df,current_df],ignore_index=True)
df3=previous_df
df3

,filerNid,electionNid,seatNid,candidateNid,committeeName,election_name
0,152470251,152889535,None,0,Oaklanders for Good Government & Safe Neighbor...,11/4/2014 - General
1,152470251,152889535,None,0,Oaklanders for Good Government & Safe Neighbor...,11/4/2014 - General
2,152248473,152889535,None,0,"Neighbors for a Safer Oakland 2014, Yes on Z, ...",11/4/2014 - General
3,176200973,155034677,None,0,J.R. 'Eddie' Orton III and Amy Orton,11/6/2018 - General
4,176200973,155034677,None,0,J.R. 'Eddie' Orton III and Amy Orton,11/6/2018 - General
...,...,...,...,...,...,...
5225,202798745,178981824,None,None,"Oaklanders Together, a coalition of small and ...",11/8/2022 - General
5226,204606835,178981824,None,None,Oakland Rising Ballot Committee supporting Mea...,11/8/2022 - General
5227,204523642,178981824,None,None,Yes on V and Q - East Bay Tenants Union PAC,11/8/2022 - General
5228,204606835,178981824,None,None,Oakland Rising Ballot Committee supporting Mea...,11/8/2022 - General


In [16]:
df=df3.merge(forms_df, on=['filerNid'])
df=final_df.merge(df, on=['candidateNid','election_name','electionNid','seatNid'])
df=df.drop_duplicates(ignore_index=True)
df

,candidateNid,candidateName,seatNid,officeName,electionNid,election_name,filerNid,committeeName,fppc_id
0,180842909,"Davis, Benjamin ""Sam""",207972561,Director - Oakland Unified School District - 1,200879019,11/5/2024 - General,208804932,Davis for Oakland School Board 2024,1464418
1,161700164,"Fife, Carroll",207972552,City Council - City of Oakland - 3,200879019,11/5/2024 - General,208660692,Carroll Fife for City Council 2024,Pending
2,208555055,"Hirsch, Shan M",207972552,City Council - City of Oakland - 3,200879019,11/5/2024 - General,208555064,"Hirsch, Shan M",None
3,208504223,"Logan, Warren",207972552,City Council - City of Oakland - 3,200879019,11/5/2024 - General,208504236,Warren Logan for Oakland City Council District...,Pending
4,186107958,"Taylor, Faye",207972552,City Council - City of Oakland - 3,200879019,11/5/2024 - General,209649414,"Taylor, Faye",None
...,...,...,...,...,...,...,...,...,...
126,190793822,"Tapscott, Ben",172302093,Director - Oakland Unified School District - 7,165189423,11/3/2020 - General,190793841,"Ben ""Coach"" Tapscott for Oakland School Board ...",1430904
127,191296935,"Taylor, Bronche Jerard",172302093,Director - Oakland Unified School District - 7,165189423,11/3/2020 - General,191296956,Committee to Support Bronche Taylor for OUSD S...,1428595
128,181562200,"Thompson, Clifford",172302093,Director - Oakland Unified School District - 7,165189423,11/3/2020 - General,181562231,CLIFFORD THOMPSON OAKLAND UNIFIED SCHOOL DISTR...,1427679
129,183504991,"Valerio, Victor",172302093,Director - Oakland Unified School District - 7,165189423,11/3/2020 - General,183505003,VALERIO FOR OUSD SCHOOL BOARD 2020,1422389


In [17]:
dfNew = final_df.merge(df3, on=['candidateNid','election_name','electionNid','seatNid'])
core_df=dfNew[['candidateName','officeName','committeeName','election_name','filerNid']]
dfdf=core_df.merge(fppc_df, on=['filerNid'])